<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 20px; height: 55px">

# Project 4: Web Scraping Job Postings

## Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:

   1. Determine the industry factors that are most important in predicting the salary amounts for these data.
   2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. 

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer these two questions.

---

## Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to answer the two questions described above.

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

### QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:
- What components of a job posting distinguish data scientists from other data jobs?
- What features are important for distinguishing junior vs. senior positions?
- Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?

You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your models' performance.


### BONUS PROBLEM

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs. Plot the ROC curve.

---

## Requirements

1. Scrape and prepare your own data.

2. **Create and compare at least two models for each section**. One of the two models should be a decision tree or ensemble model. The other can be a classifier or regression of your choosing (e.g. Ridge, logistic regression, KNN, SVM, etc).
   - Section 1: Job Salary Trends
   - Section 2: Job Category Factors

3. Prepare a polished Jupyter Notebook with your analysis for a peer audience of data scientists. 
   - Make sure to clearly describe and label each section.
   - Comment on your code so that others could, in theory, replicate your work.

4. A brief writeup in an executive summary, written for a non-technical audience.
   - Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

#### BONUS

5. Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions.

6. Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

---

## Suggestions for Getting Started

1. Collect data from [Indeed.com](www.indeed.com) (or another aggregator) on data-related jobs to use in predicting salary trends for your analysis.
  - Select and parse data from *at least 1000 postings* for jobs, potentially from multiple location searches.
2. Find out what factors most directly impact salaries (e.g. title, location, department, etc).
  - Test, validate, and describe your models. What factors predict salary category? How do your models perform?
3. Discover which features have the greatest importance when determining a low vs. high paying job.
  - Your Boss is interested in what overall features hold the greatest significance.
  - HR is interested in which SKILLS and KEY WORDS hold the greatest significance.   
4. Author an executive summary that details the highlights of your analysis for a non-technical audience.
5. If tackling the bonus question, try framing the salary problem as a classification problem detecting low vs. high salary positions.

---

## Useful Resources

- Scraping is one of the most fun, useful and interesting skills out there. Donâ€™t lose out by copying someone else's code!
- [Here is some advice on how to write for a non-technical audience](http://programmers.stackexchange.com/questions/11523/explaining-technical-things-to-non-technical-people)
- [Documentation for BeautifulSoup can be found here](http://www.crummy.com/software/BeautifulSoup/).

---

### Project Feedback + Evaluation

For all projects, students will be evaluated on a simple 3 point scale (0, 1, or 2). Instructors will use this rubric when scoring student performance on each of the core project **requirements:** 

Score | Expectations
----- | ------------
**0** | _Does not meet expectations. Try again._
**1** | _Meets expectations. Good job._
**2** | _Surpasses expectations. Brilliant!_

[For more information on how we grade our DSI projects, see our project grading walkthrough.](https://git.generalassemb.ly/dsi-projects/readme/blob/master/README.md)

In [1]:
from bs4 import BeautifulSoup
import urllib2
import pickle
from time import sleep
import requests
import re
import pandas as pd

In [3]:
url = "https://www.indeed.com.sg/m/jobs?q=Data&l=Singapore&start=0"
response = urllib2.urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
page_links = []
weblink = 'https://www.indeed.com.sg/'

for link in soup.findAll('a', attrs={'href': re.compile("^viewjob?")}):
    page_link = weblink+link.get('href')
    page_links.append(page_link)

page_links

[u'https://www.indeed.com.sg/viewjob?jk=c25e0b0398772de7',
 u'https://www.indeed.com.sg/viewjob?jk=824ec4a978ab8642',
 u'https://www.indeed.com.sg/viewjob?jk=1f1e8dd98e118f2c',
 u'https://www.indeed.com.sg/viewjob?jk=8272d375c05e9707',
 u'https://www.indeed.com.sg/viewjob?jk=16d91aa9466a97e8',
 u'https://www.indeed.com.sg/viewjob?jk=24b018ac410e0732',
 u'https://www.indeed.com.sg/viewjob?jk=d4cbc5dfd918d49d',
 u'https://www.indeed.com.sg/viewjob?jk=0ee8287e82a8e6bc',
 u'https://www.indeed.com.sg/viewjob?jk=5295c09e61003f46',
 u'https://www.indeed.com.sg/viewjob?jk=21641976eda2f1a1']

### Scraping

The functions below are functions that scrape job titles, job links, salaries, employers and job descriptions from the Indeed website.

They are helper functions that will be used in a master function.

In [5]:
def extract_job_title(soup): 
    
    job_list = []
        
    for item in soup.find_all(name='h2', attrs={'class':'jobTitle'}):
        
        try:
            job_list.append(item.text.strip(':'))
        
        except:
        
            job_list.append('Scraping unsuccessful')
            continue
        
    return job_list

def extract_job_link(soup):
    
    page_links = []
    
    weblink = 'https://www.indeed.com.sg/'
            
    for link in soup.findAll('a', attrs={'href': re.compile("^viewjob?")}):
        try:
            page_link = weblink+link.get('href')
            page_links.append(page_link)
            
        except:
            page_links.append('Scraping unsuccessful')
            continue
        
    return page_links

def extract_salary(soup):
    
    mobile_page_links = []
    salary_list = []
    
    weblink = 'https://www.indeed.com.sg/m/'

    for link in soup.findAll('a', attrs={'href': re.compile("^viewjob?")}):
        mobile_page_link = weblink+link.get('href')
        mobile_page_links.append(mobile_page_link)

    for mobile_link in mobile_page_links:
        
        try:
            response_mobile = urllib2.urlopen(mobile_link)
            html_mobile = response_mobile.read()
            soup_mobile = BeautifulSoup(html_mobile, 'html.parser')
            salary_mobile = soup_mobile.find('span',{'class':'salary'})

            if salary_mobile > 0:
                salary_list.append(salary_mobile.text)
            else:
                salary_list.append('Salary not stated')
                
        except:
            salary_list.append('Scraping Unsuccessful')
            continue
        
    return salary_list
    
def extract_employer(links):
    
    employers = []
    
    for link in links:
        try:
            response = urllib2.urlopen(link)
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            employer = soup.find('span',{'class':'company'})

            if employer > 0:
                employers.append(employer.text)   
            else:
                employers.append('Employer not stated')
        except:
            employers.append('Scraping Unsuccessful')
            continue
            
    return employers

def extract_job_description(links):
    
    job_description = []
    
    for link in links:
        try:
            response = urllib2.urlopen(link)
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            job_summary = soup.find('span',{'id':'job_summary'})

            if len(job_summary) > 0:
                job_description.append(job_summary.text)   
            else:
                job_description.append('Job description not stated')
                
        except:
            job_description.append('Scraping Unsuccessful')
            continue
            
    return job_description

In [9]:
# Master Function 1. The result is a dataframe.

def indeed_info_scraper(base_url):
    
    cap=1000
        
    pages_to_scrape = range(0,cap,10)
    indeed_pages = []
    indeed_url = []
    
    job_titles = []
    job_links = []
    employers = []
    salaries = []
        
    for x in pages_to_scrape:
        indeed_pages.append(str(x))

    for page in indeed_pages:
        indeed_url.append(base_url+page)
    
    for url in indeed_url:
        
        response = urllib2.urlopen(url)
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        job_titles.extend(extract_job_title(soup))
        job_links.extend(extract_job_link(soup))
        salaries.extend(extract_salary(soup))
    
    employers.extend(extract_employer(job_links))

    job_titles_df = pd.DataFrame(job_titles,columns=['Job Title'])
    job_links_df = pd.DataFrame(job_links,columns=['Job Link'])
    
    employers_df = pd.DataFrame(employers,columns=['Employer'])
        
    salaries_df = pd.DataFrame(salaries,columns=['Salary'])
    
    indeed_scraped_df = pd.concat([job_titles_df,job_links_df,employers_df,
                                   salaries_df],axis=1)
    
    return indeed_scraped_df


In [10]:
# Master Function 2. A different function is used for scraping job descriptions
# because of errors that could not be easily resolved.

def indeed_job_description_scraper(base_url):
    
    cap=1000
        
    pages_to_scrape = range(0,cap,10)
    indeed_pages = []
    indeed_url = []
    
    job_links = []
    job_summaries = []
        
    for x in pages_to_scrape:
        indeed_pages.append(str(x))

    for page in indeed_pages:
        indeed_url.append(base_url+page)
    
    for url in indeed_url:
        
        response = urllib2.urlopen(url)
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        job_links.extend(extract_job_link(soup))
        
    job_summaries.extend(extract_job_description(job_links))
        
    job_summaries_df = pd.DataFrame(job_summaries,columns=['Job Description'])
    
    return job_summaries_df

In [11]:
base_url = 'https://www.indeed.com.sg/m/jobs?q=Data&l=Singapore&start='

In [12]:
# Scraping for information

indeed_scraped_df = indeed_info_scraper(base_url)

In [13]:
# Scraping for job descriptions

job_descriptions_df = indeed_job_description_scraper(base_url)

In [14]:
indeed_data_df = pd.concat([indeed_scraped_df,job_descriptions_df],axis=1)

In [15]:
# Scraped dataframe with 1000 values as required by question.

indeed_data_df.shape

(1000, 5)

In [16]:
pickle_out = open("indeed_data_df.pickle","wb")
pickle.dump(indeed_data_df, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("indeed_data_df.pickle","rb")
indeed_data_df = pickle.load(pickle_in)